In [46]:
import numpy as np

In [48]:
class fileClass:
    f = None
    array = []

    def __init__(path, self):
        self.f = open(path, "r")
        
    def readFile(self): 
        while (self.f):
            line = self.f.read(1)
            self.array.append(line)
            if line == '':
                break;